In [5]:
import pandas as pd
from preProccesPuzzle import PreProcess
from clue_classifier import ClueClassifier
from constraints import Constraint, IdentityConstrain, NextToConstrain, DistanceConstrain,RightConstrain, LeftConstrain, DirectRightConstrain, DirectLeftConstrain, PositionAbsoluteConstrain, PositionAbsoluteNegativeConstrain
from constraint_solver import ConstraintSolver
from concurrent.futures import ProcessPoolExecutor


In [6]:
def constraint_factory(attrs, clues):
    constrains: list[Constraint] = []
    classifier = ClueClassifier()
    for c in clues:
        clue, clue_type = classifier.classify(c)

        if clue_type == "IDENTITY":
            constrains.append(IdentityConstrain(attrs, clue))
        if clue_type == "NEXT_TO":
            constrains.append(NextToConstrain(attrs, clue))
        if clue_type == "LEFT":
            constrains.append(LeftConstrain(attrs, clue))
        if clue_type == "RIGHT":
            constrains.append(RightConstrain(attrs, clue))
        if clue_type == "DISTANCE":
            constrains.append(DistanceConstrain(attrs, clue))
        if clue_type == "DIRECT_LEFT":
            constrains.append(DirectLeftConstrain(attrs, clue))
        if clue_type == "DIRECT_RIGHT":
            constrains.append(DirectRightConstrain(attrs, clue))
        if clue_type == "POSITION_ABSOLUTE":
            constrains.append(PositionAbsoluteConstrain(attrs, clue))
        if clue_type == "POSITION_ABSOLUTE_NEGATIVE":
            constrains.append(PositionAbsoluteNegativeConstrain(attrs, clue))
        if clue_type == "UNKNOWN":
            raise TypeError
    
    return constrains

In [7]:
gridmode = pd.read_parquet("Gridmode-00000-of-00001.parquet")
mc = pd.read_parquet("mc-00000-of-00001.parquet")
ppp = PreProcess()

In [8]:
puzzle = gridmode.puzzle.iloc[337].lower()

attrs, clues = ppp.proccess(puzzle)

print(puzzle)

constrains: list[Constraint] = constraint_factory(attrs, clues)

Cs = ConstraintSolver(attrs, constrains)

for c in constrains:
    print(c.get_info())

solution = Cs.solve()

Cs.print_solution(solution)

there are 4 houses, numbered 1 to 4 from left to right, as seen from across the street. each house is occupied by a different person. each house has a unique attribute for each of the following characteristics:
 - each person has a unique name: `alice`, `eric`, `peter`, `arnold`
 - each person has a unique level of education: `master`, `bachelor`, `associate`, `high school`
 - people have unique hair colors: `black`, `red`, `brown`, `blonde`
 - each person has a unique type of pet: `bird`, `cat`, `fish`, `dog`
 - everyone has something unique for lunch: `grilled cheese`, `spaghetti`, `pizza`, `stew`
 - the people keep unique animals: `horse`, `cat`, `fish`, `bird`

## clues:
1. eric is the person with an associate's degree.
2. the person who keeps a pet bird is in the third house.
3. the person with a master's degree is not in the first house.
4. the person who has a cat is the bird keeper.
5. there is one house between the person who keeps horses and the person with an aquarium of fis